In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("US_Accidents_Analysis") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()


25/07/08 03:19:43 WARN Utils: Your hostname, DESKTOP-DD5N9SQ resolves to a loopback address: 127.0.1.1; using 172.30.167.134 instead (on interface eth0)
25/07/08 03:19:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/08 03:20:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df_final = spark.read.parquet("data/df_final.parquet")


In [4]:
# Renommer la colonne "Severity" en "label" AVANT le split
df_final = df_final.withColumnRenamed("Severity", "label")

# (Si tu as déjà fait le split, refais-le après le renommage)
train_data, test_data = df_final.randomSplit([0.8, 0.2], seed=42)


In [5]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# 1. Définir le modèle
lr = LogisticRegression(featuresCol="features", labelCol="label", maxIter=10)

# 2. Entraîner le modèle
lr_model = lr.fit(train_data)

# 3. Faire des prédictions
predictions = lr_model.transform(test_data)

# 4. Évaluer avec l'accuracy
evaluator = MulticlassClassificationEvaluator(
    labelCol="label",
    predictionCol="prediction",
    metricName="accuracy"
)

accuracy = evaluator.evaluate(predictions)
print(f"✅ Accuracy: {accuracy:.4f}")


25/07/08 03:23:36 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
25/07/08 03:23:36 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS
[Stage 15:==============================================>           (4 + 1) / 5]

✅ Accuracy: 0.9169


In [6]:
# Afficher un échantillon de 10 prédictions avec détails
predictions.select("features", "label", "prediction", "probability").show(10, truncate=False)


25/07/08 03:28:00 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
[Stage 17:>                                                         (0 + 1) / 1]

+------------------------------------------------------------------------------------+-----+----------+---------------------------------------------------------------------------------------------------------+
|features                                                                            |label|prediction|probability                                                                                              |
+------------------------------------------------------------------------------------+-----+----------+---------------------------------------------------------------------------------------------------------+
|[28.9,20.4,100.0,30.19,0.8,9.2,0.01,0.01,55.0,47.0,5.0,15.0,8.0,1.0]                |1    |2.0       |[2.9667021141083517E-7,6.825624351801085E-4,0.9193214615575306,0.05418348848106185,0.02581219085601594]  |
|[52.0,52.0,50.0,29.42,10.0,6.0,0.0,0.855,31.0,28.0,21.0,9.0,0.0,0.0]                |1    |2.0       |[4.0220870120860637E-7,0.0012989568151752836,0.8822447007

In [8]:
# Filtrer où prediction == label
correct_preds = predictions.filter(predictions.prediction == predictions.label)

# Afficher 10 lignes
correct_preds.select("features", "label", "prediction", "probability").show(10, truncate=False)


[Stage 24:>                                                         (0 + 1) / 1]

+-------------------------------------------------------------------------+-----+----------+---------------------------------------------------------------------------------------------------------+
|features                                                                 |label|prediction|probability                                                                                              |
+-------------------------------------------------------------------------+-----+----------+---------------------------------------------------------------------------------------------------------+
|[21.9,9.8,92.0,29.71,2.0,12.7,0.01,0.01,58.0,5.0,19.0,6.0,8.0,1.0]       |2    |2.0       |[3.3597154327465493E-7,7.179524448202113E-4,0.8933826876629855,0.07646553647433249,0.029433487446318414] |
|[15.4,1.0,82.0,29.95,7.0,13.8,0.0,0.01,3322.0,98.0,19.0,7.0,8.0,1.0]     |2    |2.0       |[4.52549375814635E-7,8.727129137294235E-4,0.835761532968183,0.09417584802002753,0.06918945354868408]     |
|[6.1

In [7]:
# Nombre d'exemples correctement classés
correct = predictions.filter(predictions.label == predictions.prediction).count()

# Total d'exemples testés
total = predictions.count()

print(f"Précision manuelle : {correct}/{total} = {correct/total:.4f}")


[Stage 21:==============================================>           (4 + 1) / 5]

Précision manuelle : 649027/707880 = 0.9169


In [11]:
lr_model.save("models/logistic_model_us_accidents")


In [12]:
predictions.select("features", "label", "prediction", "probability").show(10, truncate=False)
# Exemple : prendre 5 lignes de test_data
some_data = test_data.limit(5)

# Faire des prédictions
some_predictions = lr_model.transform(some_data)
some_predictions.select("features", "label", "prediction", "probability").show(truncate=False)


+------------------------------------------------------------------------------------+-----+----------+---------------------------------------------------------------------------------------------------------+
|features                                                                            |label|prediction|probability                                                                                              |
+------------------------------------------------------------------------------------+-----+----------+---------------------------------------------------------------------------------------------------------+
|[28.9,20.4,100.0,30.19,0.8,9.2,0.01,0.01,55.0,47.0,5.0,15.0,8.0,1.0]                |1    |2.0       |[2.966702114108357E-7,6.825624351801061E-4,0.9193214615575306,0.05418348848106207,0.025812190856015802]  |
|[52.0,52.0,50.0,29.42,10.0,6.0,0.0,0.855,31.0,28.0,21.0,9.0,0.0,0.0]                |1    |2.0       |[4.0220870120860537E-7,0.0012989568151752793,0.8822447007

[Stage 25:>                                                         (0 + 1) / 1]

+----------------------------------------------------------------------+-----+----------+---------------------------------------------------------------------------------------------------------+
|features                                                              |label|prediction|probability                                                                                              |
+----------------------------------------------------------------------+-----+----------+---------------------------------------------------------------------------------------------------------+
|[28.9,20.4,100.0,30.19,0.8,9.2,0.01,0.01,55.0,47.0,5.0,15.0,8.0,1.0]  |1    |2.0       |[2.966702114108357E-7,6.825624351801061E-4,0.9193214615575306,0.05418348848106207,0.025812190856015802]  |
|[52.0,52.0,50.0,29.42,10.0,6.0,0.0,0.855,31.0,28.0,21.0,9.0,0.0,0.0]  |1    |2.0       |[4.0220870120860537E-7,0.0012989568151752793,0.8822447007205643,0.08657395890241448,0.029881981353144783]|
|[50.0,50.0,87.0,27.

## 📊 Modèle de Régression Logistique avec PySpark

### 🔧 Préparation des données
- Nettoyage d'un jeu de données de près de 4 millions de lignes.
- Encodage des variables catégorielles à l’aide de `StringIndexer`.
- Assemblage des variables numériques et indexées avec `VectorAssembler`.

### 🧠 Entraînement du modèle
- Utilisation d’un **modèle de régression logistique** via `pyspark.ml.classification.LogisticRegression`.
- Le modèle a été entraîné sur un jeu de données d'entraînement issu d’un `randomSplit` (80% entraînement, 20% test).

### ✅ Résultats
- **Accuracy obtenue** sur le jeu de test : **91,69%**
- Test sur un petit échantillon (5 lignes) :
  - Les prédictions sont majoritairement correctes.
  - Les probabilités montrent une forte confiance du modèle dans ses décisions.

### 📌 Conclusion
- La régression logistique s’est montrée efficace pour ce problème de classification.
- Ce résultat confirme l'importance du **prétraitement rigoureux** et du **choix des variables** dans la réussite du modèle.



In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# ⚙️ Initialiser avec maxBins augmenté
dt = DecisionTreeClassifier(featuresCol="features", labelCol="label", maxDepth=10, maxBins=13000)

# 🧠 Entraîner
dt_model = dt.fit(train_data)

# 🔮 Prédictions
dt_predictions = dt_model.transform(test_data)

# ✅ Accuracy
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
dt_accuracy = evaluator.evaluate(dt_predictions)

print(f"✅ Decision Tree Accuracy: {dt_accuracy:.4f}")


[Stage 30:>                                                         (0 + 4) / 5]

In [15]:
dt_model.save("models/decision_tree_model")


In [16]:
from pyspark.ml.classification import RandomForestClassifier

# ⚙️ Créer le modèle avec maxBins suffisamment élevé
rf = RandomForestClassifier(featuresCol="features", labelCol="label", numTrees=50, maxDepth=10, maxBins=13000)

# 🧠 Entraîner
rf_model = rf.fit(train_data)

# 🔮 Prédictions
rf_predictions = rf_model.transform(test_data)

# ✅ Accuracy
rf_accuracy = evaluator.evaluate(rf_predictions)
print(f"✅ Random Forest Accuracy: {rf_accuracy:.4f}")

# 💾 Sauvegarde
rf_model.save("models/random_forest_model")


25/07/07 01:05:45 WARN MemoryStore: Not enough space to cache rdd_208_3 in memory! (computed 151.6 MiB so far)
25/07/07 01:05:45 WARN MemoryStore: Not enough space to cache rdd_208_2 in memory! (computed 151.6 MiB so far)
25/07/07 01:05:45 WARN BlockManager: Persisting block rdd_208_2 to disk instead.
25/07/07 01:05:45 WARN BlockManager: Persisting block rdd_208_3 to disk instead.
25/07/07 01:05:45 WARN MemoryStore: Not enough space to cache rdd_208_0 in memory! (computed 151.6 MiB so far)
25/07/07 01:05:45 WARN BlockManager: Persisting block rdd_208_0 to disk instead.
25/07/07 01:07:05 WARN DAGScheduler: Broadcasting large task binary with size 2.2 MiB
25/07/07 01:08:25 WARN DAGScheduler: Broadcasting large task binary with size 4.5 MiB
25/07/07 01:10:38 WARN DAGScheduler: Broadcasting large task binary with size 8.2 MiB
25/07/07 01:13:28 WARN DAGScheduler: Broadcasting large task binary with size 12.3 MiB
25/07/07 01:17:39 WARN DAGScheduler: Broadcasting large task binary with size 9

✅ Random Forest Accuracy: 0.9210


25/07/07 15:47:41 WARN TaskSetManager: Stage 203 contains a task of very large size (12011 KiB). The maximum recommended task size is 1000 KiB.
                                                                                

In [17]:
dt_model.save("models/decision_tree_model_2")


## 🤖 Comparaison des Modèles de Classification (Logistic Regression, Decision Tree, Random Forest)

Dans notre projet d’analyse des accidents de la route, nous avons testé plusieurs modèles de classification supervisée afin de prédire la **gravité des accidents** (`Severity`), une variable numérique que nous avons traitée comme une **étiquette (`label`) catégorielle**.

### 🎯 Objectif :
Prédire le **niveau de gravité d’un accident** (faible à élevé) en se basant sur les caractéristiques comme l’heure, le lieu, les conditions météo, etc.

---

### 📌 Modèle 1 : **Logistic Regression**

- **Type** : Modèle linéaire probabiliste  
- **Principe** : Apprend une frontière de décision linéaire entre les classes.  
- **Avantage** : Rapide, efficace sur des données peu complexes.  
- **Résultat** :  
  `✅ Accuracy: 0.9169`

---

### 📌 Modèle 2 : **Decision Tree Classifier**

- **Type** : Arbre de décision unitaire  
- **Principe** : Divise les données en fonction des attributs en suivant les valeurs qui réduisent l'entropie.  
- **Avantage** : Facile à interpréter, fonctionne bien avec des données non linéaires.  
- **Résultat** :  
  `✅ Accuracy: 0.9214`

---

### 📌 Modèle 3 : **Random Forest Classifier** *(à venir)*

- **Type** : Ensemble d’arbres (bagging)  
- **Principe** : Combine plusieurs arbres de décision aléatoires pour améliorer la stabilité et la précision.  
- **Avantage** : Réduit le surapprentissage (overfitting), robuste.  
- **Résultat** : *(sera mesuré)*
 `✅ Accuracy: 0.9210`

---

### 📈 Prédictions réalisées

Chaque modèle a été entraîné sur un jeu de données d’accidents, avec des variables transformées (ex. : `features`, `label`), puis évalué sur des données de test.  
Les prédictions permettent de savoir **si un accident est susceptible d’être grave ou non** en fonction des conditions détectées (météo, route, etc.).

Exemple de prédiction (Logistic Regression) :


AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/home/souhail/bigdata-accidents/notebooks/data/accidents_preprocessed.parquet.

In [10]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import col

# 1. Initialiser le modèle Random Forest
rf = RandomForestClassifier(featuresCol="features", labelCol="label", numTrees=100, maxDepth=15, maxBins=13000)

# 2. Entraîner le modèle
rf_model = rf.fit(train_data)

# 3. Prédictions
rf_predictions = rf_model.transform(test_data)

# 4. Accuracy
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
rf_accuracy = evaluator.evaluate(rf_predictions)
print(f"🎯 Random Forest Accuracy: {rf_accuracy:.4f}")

# 5. Affichage des prédictions
rf_predictions.select("features", "label", "prediction", "probability").show(10, truncate=False)


ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/home/souhail/.local/lib/python3.10/site-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/souhail/.local/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/souhail/.local/lib/python3.10/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


Py4JError: org does not exist in the JVM